In [ ]:
# Caso não tenha instalado, instalar o pmdarima e atualizar o statsmodels
#!pip install update statsmodels==0.12.1 
#%pip install pmdarima

import pmdarima as pm

# Carrega os dados
series = pd.read_csv('dataset.csv')

# Se necessário faz uma separação
df = series[:375]

# Calcula o melhor conjunto de parâmetros para o modelo ARIMA
pm.auto_arima(df['coluna'], start_p=0, start_q=0, max_q=6, seasonal=False, trace=True, maxiter=1000)

In [ ]:
# Previsão com o modelo Arima multi-step multi-output
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA
import numpy
from math import sqrt
from sklearn.metrics import mean_squared_error
from pandas.plotting import autocorrelation_plot
from pandas.plotting import lag_plot
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

# Carrega os dados
series = read_csv('dataset.csv', header=0, index_col=0, squeeze=True, parse_dates=True)

# Visualiza o gráfico de autocorrelação, autocorrelação parcial e por último o padrão dos dados (linear ou não linear)
autocorrelation_plot(series)
plot_pacf(series)
lag_plot(series)

# Divide o conjunto e passa para X apenas os dados correspondentes ao conjunto de treinamento
X = series.values[:375]

# Faz o fit model
model = ARIMA(X, order=(6,0,3))
model_fit = model.fit()

# Faz a previsão de multiplos passos fora da amostra 
forecast = model_fit.forecast(steps=125, dynamic=True)

# Calcula o erro de previsão através da métrica RMSE entre o conjunto de teste e os dados previstos
rmse = sqrt(mean_squared_error(series[375:], forecast))
print('Test RMSE: %.3f' % rmse)

In [ ]:
# ESTE TRECHO DE CÓDIGO CALCULA O COEFICIENTE DE AUTOCORRELAÇÃO DAS OBSERVAÇÕES (O PADRÃO É PEARSON)
from pandas import DataFrame

teste = pd.read_csv('dataset.csv')

test = DataFrame(teste)

test.info()

shift_1 = test['coluna'].autocorr(lag=1)
print(shift_1)

In [ ]:
# PREVISÃO COM O MODELO LSTM UNIVARIADO -- PREVISÃO DE MÚLTIPLAS ETAPAS COM SAÍDA VETORIAL, OU SEJA, MULTI-OUTPUT
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
from pandas import Series
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.layers import Dropout
#from keras.regularizers import l2
from keras.regularizers import l1
import matplotlib.pyplot as plt


# Divide a sequencia em amostras
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# Carrega os dados e e passa para o df apenas os dados de treinamento
series = pd.read_csv('dataset.csv', header=0, index_col=0, squeeze=True, parse_dates=True)
df = series[:375]
raw_seq = df.values

# Escolhe o número de passos de entrada e de saída
n_steps_in, n_steps_out = 3, 125

# Divide em amostras para o modelo
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

# Faz o reshape [amostra, passos no tempo] into [amostra, passos no tempo, nº caracteristicas/recursos/variáveis]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# Define o modelo, neste caso empilhado, pois são duas camadas LSTM
model = Sequential()
model.add(LSTM(5, return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(5, dropout=0.2, activation='tanh'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mean_squared_error')

# Ajusta o modelo
model.fit(X, y, epochs=100, verbose=0, batch_size=12)

# Carrega os valores para a entrada, neste caso foram definidos os três primeiros
x_input = raw_seq[:3]
x_input = x_input.reshape((1, n_steps_in, n_features))

# Faz as previsões
yhat = model.predict(x_input, verbose=0)

# Usa o flatten para conseguir fazer o cálculo do erro de previsão
predictions = yhat.flatten().tolist()
#print(len(predictions))

# Calcula o erro de previsão baseado na métrica RMSE. Aqui é comparado o conjunto de teste com as previsões feitas pelo modelo
rmse = sqrt(mean_squared_error(series[375:], predictions))

# Exibe o erro
print('Test RMSE: %.3f' % rmse)

# Plota o resultado, ou seja, o cojunto de teste x as previsões
dias = range(0,125)
plt.plot(dias, series[2931:], dias, predictions)
plt.legend(['Real','Previsto'])
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

# Aqui salva a imagem em formato eps
plt.savefig('Plotsaida.eps', format='eps')